In [2]:
!pip install flask spacy transformers pydantic
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 83.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
import spacy

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Example input
input_text = "Create a service to add an order"

# Parse the input text
doc = nlp(input_text)

# Extract important parts (e.g., action, entity)
for token in doc:
    print(token.text, token.dep_, token.pos_)


Create ROOT VERB
a det DET
service dobj NOUN
to aux PART
add advcl VERB
an det DET
order dobj NOUN


In [6]:
%%writefile app.py
import streamlit as st
import spacy

# Load the English NLP model
nlp = spacy.load("en_core_web_sm")

def parse_request(request_text):
    doc = nlp(request_text)
    action, entity = None, None
    for token in doc:
        if token.dep_ == "ROOT":
            action = token.lemma_  # Main verb
        if token.pos_ == "NOUN":
            entity = token.text  # Extract noun as entity
    return action, entity

def generate_flask_service(entity_name, action):
    code = f"""
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/{entity_name}', methods=['POST'])
def {action}_{entity_name}():
    data = request.get_json()
    return jsonify(data), 201

if __name__ == '__main__':
    app.run(debug=True)
"""
    return code

# Streamlit app interface
st.title("AI-Powered Service Generator")

request_text = st.text_input("Enter your service request (e.g., 'Create a service to add an order'): ")

if st.button("Generate Service Code"):
    action, entity = parse_request(request_text)
    if action and entity:
        service_code = generate_flask_service(entity, action)
        st.code(service_code, language='python')
    else:
        st.error("Could not parse the request. Please try again.")


Writing app.py
